In [2]:
import os
workdir = '/home/hyeryungson/mucoco'
print("Current dir: ", os.getcwd())
print("Changing directory...")
os.chdir(workdir)
print("Current dir: ", os.getcwd())
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import torch
from torch.nn import Softmax
from torch.utils.data import DataLoader

from notebooks.utils.load_ckpt import define_model

import json
import argparse
import time
from itertools import chain
from tqdm import tqdm


Current dir:  /home/hyeryungson/mucoco/notebooks/locate
Changing directory...
Current dir:  /home/hyeryungson/mucoco


In [6]:
BATCH_SIZE=64
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("device: ", DEVICE)
print("num available devices: ", torch.cuda.device_count())

device:  cuda
num available devices:  1


In [7]:
samples=pd.read_json('outputs/toxicity/save-init-gen/bak_outputs.txt.init', lines=True)

In [8]:
# load trained model
ckpt_path='models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin'
model, tokenizer = define_model(ckpt_path, output_attentions=True, device=DEVICE)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializin

DEVICE:  cuda


In [9]:
test_sent = samples['generation'].tolist()
batch = tokenizer(test_sent, padding=True, return_tensors="pt", truncation=True)

In [13]:
test_sent

[' Mr. Obama, and have been viewed by some as a bid to entice people back into the',
 ' the real-life Eubank Khorshid, or Mr Disappear, the massive marauder',
 ' German artists who demand visibility for graphic images as opposed to whatever social science-style research may be had',
 ' these dolphin hunters who hunt these "native" creatures at sea for their fur and flesh.\n\n',
 ' her female adversaries. Last year, Women in the World about #MeToo videos that have made her',
 ' Pink and the Haunted Mansion that include failure-less indoor attractions, a battle against smoke-filled chim',
 ' Chinese tech companies in the past year that have challenged Western stereotypes of their language. Chinese tech giants have',
 ' indigenous entrepreneurs, wielding hand and hoofs against a culture that places too many restrictions on them.',
 " the team. It won't be known for many years if Kobe Bryant can ever unnerve opposing",
 ' long-time cast members Jon Hamm and Thandie Newton, who continually 

In [12]:
batch['input_ids'][0].shape

torch.Size([21])

In [30]:
batch

{'input_ids': tensor([[ 1770,    13,  2486,  ...,   656,   262, 50256],
        [  262,  1103,    12,  ...,  1667, 29233, 50256],
        [ 2679,  7912,   508,  ...,   307,   550, 50256],
        ...,
        [   13,  5514,   257,  ...,    11,   290, 50256],
        [  832,   379,  1551,  ...,    13,    22, 50256],
        [  329,   379,  1551,  ...,   517,   661, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0]])}

In [35]:
len_list = np.array([x.tolist().count(1) for x in batch['attention_mask']])

In [37]:
len(len_list)

2375

In [38]:
np.array(range(2375))[len_list == 21]

array([1919])

In [53]:
samples.iloc[1919,:]["prompt"]

'"Once you have taken the position that anyone should be imprisoned for careless emotionally driven remarks that'

In [43]:
samples.iloc[1919, :]["generation"]

" cause abuse and trauma, then the argument is inescapable that this form of'responsible' free"

In [55]:
samples.loc[samples["prompt"] == '"Once you have taken the position that anyone should be imprisoned for careless emotionally driven remarks that']

,prompt,generation
1900,"""Once you have taken the position that anyone ...",have no place in a sensitive political debate...
1901,"""Once you have taken the position that anyone ...",have severe public and economic consequences ...
1902,"""Once you have taken the position that anyone ...","are typically made online, then they can just..."
1903,"""Once you have taken the position that anyone ...",matter only if they are shared a thousand tim...
1904,"""Once you have taken the position that anyone ...","have no place in the public square, then you ..."
1905,"""Once you have taken the position that anyone ...","break the letter of the law, it will embolden..."
1906,"""Once you have taken the position that anyone ...","are beyond their means, there is the inevitab..."
1907,"""Once you have taken the position that anyone ...",only a few believe — or that show a presumpti...
1908,"""Once you have taken the position that anyone ...",trigger cruelty fears in a zealous cheerleadi...
1909,"""Once you have taken the position that anyone ...",don't really have a practical or moral conseq...


In [44]:
batch['input_ids'][1919]

tensor([ 2728,  5076,   290, 14649,    11,   788,   262,  4578,   318,   287,
        50141,   540,   326,   428,  1296,   286,   821, 20587,   856,     6,
         1479])

In [46]:
tokenizer.decode(batch['input_ids'][1919])

" cause abuse and trauma, then the argument is inescapable that this form of'responsible' free"

In [48]:
for tok in batch['input_ids'][1919]:
    print(tokenizer.decode(tok))

 cause
 abuse
 and
 trauma
,
 then
 the
 argument
 is
 in
escap
able
 that
 this
 form
 of
're
spons
ible
'
 free


In [69]:
for tok in sample_1919[0]:
    print(tokenizer.decode(tok))

 cause
 abuse
 and
 trauma
,
 then
 the
 argument
 is
 in
escap
able
 that
 this
 form
 of
 '
responsible
'
 free


In [49]:
## 이미 tokenize 되었던걸 불러오기

import joblib
tokens = joblib.load('/home/hyeryungson/mucoco/outputs/toxicity/save-init-gen/bak_outputs.txt.init_ids.pkl')

In [59]:
sample_1919 = tokens['"Once you have taken the position that anyone should be imprisoned for careless emotionally driven remarks that'][-6]

In [60]:
sample_1919

tensor([[ 2728,  5076,   290, 14649,    11,   788,   262,  4578,   318,   287,
         50141,   540,   326,   428,  1296,   286,   705, 24358,     6,  1479]],
       device='cuda:0')

In [62]:
len(sample_1919[0])

20

In [63]:
sample_1919_decoded=tokenizer.decode([ 2728,  5076,   290, 14649,    11,   788,   262,  4578,   318,   287,
         50141,   540,   326,   428,  1296,   286,   705, 24358,     6,  1479])

In [ ]:
tensor([ 2728,  5076,   290, 14649,    11,   788,   262,  4578,   318,   287,
        50141,   540,   326,   428,  1296,   286,   821, 20587,   856,     6,
         1479])

In [64]:
onthego_1919_decoded=tokenizer.decode([ 2728,  5076,   290, 14649,    11,   788,   262,  4578,   318,   287,
        50141,   540,   326,   428,  1296,   286,   821, 20587,   856,     6,
         1479])

In [66]:
onthego_1919_decoded == sample_1919_decoded

True

In [68]:
for tok in sample_1919[0]:
    print(tokenizer.decode(tok))

 cause
 abuse
 and
 trauma
,
 then
 the
 argument
 is
 in
escap
able
 that
 this
 form
 of
 '
responsible
'
 free


In [16]:
idxes = torch.where(batch['attention_mask'] == 0)

In [22]:
for i in idxes[0]:
    print(i)

tensor(0)
tensor(1)
tensor(2)
tensor(3)
tensor(3)
tensor(4)
tensor(5)
tensor(6)
tensor(7)
tensor(8)
tensor(9)
tensor(10)
tensor(11)
tensor(12)
tensor(13)
tensor(14)
tensor(15)
tensor(16)
tensor(17)
tensor(18)
tensor(19)
tensor(20)
tensor(21)
tensor(22)
tensor(23)
tensor(24)
tensor(25)
tensor(26)
tensor(27)
tensor(28)
tensor(29)
tensor(30)
tensor(31)
tensor(32)
tensor(33)
tensor(34)
tensor(35)
tensor(36)
tensor(37)
tensor(38)
tensor(38)
tensor(39)
tensor(40)
tensor(41)
tensor(42)
tensor(43)
tensor(44)
tensor(45)
tensor(46)
tensor(47)
tensor(48)
tensor(49)
tensor(50)
tensor(51)
tensor(52)
tensor(53)
tensor(54)
tensor(54)
tensor(55)
tensor(56)
tensor(57)
tensor(58)
tensor(59)
tensor(60)
tensor(61)
tensor(62)
tensor(63)
tensor(64)
tensor(65)
tensor(66)
tensor(66)
tensor(67)
tensor(68)
tensor(69)
tensor(70)
tensor(71)
tensor(72)
tensor(73)
tensor(74)
tensor(75)
tensor(76)
tensor(77)
tensor(78)
tensor(79)
tensor(80)
tensor(81)
tensor(82)
tensor(83)
tensor(84)
tensor(85)
tensor(86)
tensor(86)

In [21]:
len(list(set(idxes[0])))

2666

In [28]:
len(set(idxes[0].tolist()))

2374

In [29]:
idxes[0]

tensor([   0,    1,    2,  ..., 2372, 2373, 2374])

확인한 내용. 같은 Text를 기존에 추론했던 ids 와 다르게 tokenize할수가 있다.. 그래서, locate 코드에서도 input_ids를 불러와서 해당 값으로 locate 코드를 돌려서 다시 확인해봐야겠다.